In [1]:
# https://blog.csdn.net/marsjhao/article/details/68490105 https://blog.csdn.net/u014281392/article/details/77103747
import numpy as np
import scipy.io as sio
np.random.seed(355)  # for reproducibility
from keras.models import Sequential
from keras import layers
from keras.layers import Dropout, Bidirectional,Activation,Flatten,GRU,LSTM
from keras.layers import Dense, Input, TimeDistributed,RepeatVector
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import Model
from keras.optimizers import Adam
import math
import time
import config_APS_2 as cf
import six.moves.cPickle as pickle
from keras_self_attention import SeqSelfAttention
import keras

C:\Users\fengx\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

observation time 220751999
the number of time interval: 14
time interval: 15768000


In [2]:
# 全局变量
batch_size = 32
epochs = 20
CELL_SIZE=32
OUTPUT_SIZE=20
# input image dimensions
TIME_STEPS = cf.n_time_interval
datasetName =cf.datasetName
INPUT_SIZE = len(cf.degree_interval_list)+1
input_shape = (TIME_STEPS, INPUT_SIZE) 
print (input_shape)

(14, 16)


In [3]:
id_train, x_train, L, y_train, sz_train, time_train, vocabulary_size = pickle.load(open(cf.train_pkl, 'rb'))
id_test, x_test, L_test, y_test, sz_test, time_test, _ = pickle.load(open(cf.test_pkl, 'rb'))
id_val, x_val, L_val, y_val, sz_val, time_val, _ = pickle.load(open(cf.val_pkl, 'rb'))
for i in range(len(x_train)):
    x_train[i] = np.array(x_train[i])
for i in range(len(x_val)):
    x_val[i] = np.array(x_val[i])
for i in range(len(x_test)):
    x_test[i] = np.array(x_test[i])
x_val = np.array(x_val)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_val = np.array(y_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(y_train.shape)
# x_train = x_train.reshape(x_train.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_test = x_test.reshape(x_test.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_val = x_val.reshape(x_val.shape[0], TIME_STEPS, INPUT_SIZE, 1)

(21015, 14, 16)
(4707, 14, 16)
(4476, 14, 16)
(21015,)


In [4]:
print(np.mean(y_train))
print(np.mean(y_test))
print(np.mean(y_test) - np.mean(y_train))
# x_train = x_train.resh

2.9242036482665004
2.86929922371805
-0.05490442454845024


In [5]:
## LSTM 
model = Sequential()
model.add(GRU(units = OUTPUT_SIZE, activation='tanh', input_shape=input_shape,return_sequences=True))
model.add(GRU(units = OUTPUT_SIZE, activation='tanh',return_sequences=True))
model.add(Flatten()) #拉成一维数据
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='relu')) #全连接层2
# model.add(Dense(1, activation='relu')) #全连接层2
 
#编译模型
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
model.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000000001)
#训练模型
f = open(datasetName+'result_epoch_rnn.csv','w')
for epoch in range(epochs):
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    test_score = model.evaluate(x_test, y_test, verbose=0)
    val_score  = model.evaluate(x_val, y_val, verbose=0)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
    print('Test score:', test_score[0])
f.close()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 14, 20)            2220      
_________________________________________________________________
gru_2 (GRU)                  (None, 14, 20)            2460      
_________________________________________________________________
flatten_1 (Flatten)          (None, 280)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                4496      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 9,193
Trainable params: 9,193
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead

C:\Users\fengx\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 8s 380us/step - loss: 1.4336 - mse: 1.4336 - val_loss: 1.2816 - val_mse: 1.2816


C:\Users\fengx\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Test score: 1.2778777578896667
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 7s 332us/step - loss: 1.3339 - mse: 1.3339 - val_loss: 1.2965 - val_mse: 1.2965
Test score: 1.2845695789943765
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 7s 333us/step - loss: 1.3167 - mse: 1.3167 - val_loss: 1.2796 - val_mse: 1.2796
Test score: 1.2797020106450347
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 7s 324us/step - loss: 1.3096 - mse: 1.3096 - val_loss: 1.2932 - val_mse: 1.2932
Test score: 1.3017455953839532
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 7s 316us/step - loss: 1.3024 - mse: 1.3024 - val_loss: 1.2972 - val_mse: 1.2972
Test score: 1.3104052518567437
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [=============================

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
y_test_pre = model.predict(x_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])
f = open(datasetName+'detail_result_rnn_.csv','w')
for i in range (len(y_test)):
    f.write(str(y_test[i])+','+str(y_test_pre[i][0])+'\n')
f.close()

Test score: 1.1421112669534368
Test accuracy: 1.1421111822128296


In [9]:
## LSTM autoencoder
retrunSeq=True
input_data = Input((None,INPUT_SIZE))
encoder=GRU(units=OUTPUT_SIZE*2,  activation='tanh',return_sequences=retrunSeq, name="gru1")(input_data)
encoder=GRU(units=OUTPUT_SIZE,  activation='tanh',return_sequences=retrunSeq, name="gru2")(encoder)
encoder_out=Dense(OUTPUT_SIZE,activation='tanh')(encoder)
encoder_model = Model(inputs=input_data, outputs=encoder_out)

decoder=GRU(units=OUTPUT_SIZE, activation='tanh',return_sequences=retrunSeq,name="de_gru1")(encoder_out)
decoder=GRU(units=OUTPUT_SIZE*2,  activation='tanh', return_sequences=retrunSeq, name="de_gru2")(decoder)
decoder_out=Dense(INPUT_SIZE,activation='relu')(decoder)
autoencoder=Model(input_data,decoder_out)
 
#编译模型
autoencoder.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
autoencoder.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000001)
#训练模型
autoencoder.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, x_val), 
          shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
#预测模型
encoded_latent_train = encoder_model.predict(x_train)
encoded_latent_test = encoder_model.predict(x_test)
encoded_latent_val = encoder_model.predict(x_val)
if retrunSeq:
    encoded_latent_train=encoded_latent_train.reshape(-1,OUTPUT_SIZE*TIME_STEPS)
    encoded_latent_test=encoded_latent_test.reshape(-1,OUTPUT_SIZE*TIME_STEPS)
    encoded_latent_val=encoded_latent_val.reshape(-1,OUTPUT_SIZE*TIME_STEPS)

input_representation = Input(shape=(OUTPUT_SIZE*TIME_STEPS,))
latent_vector = Dense(16, activation='relu')(input_representation)
preddiction = Dense(1, activation='relu')(latent_vector)
model2 = Model(input= input_representation,output=preddiction)
#编译
model2.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
model2.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00001)
#训练模型
f = open('result_epoch_rnn_ae.csv','w')
for epoch in range(epochs):
    model2.fit(encoded_latent_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(encoded_latent_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    test_score = model2.evaluate(encoded_latent_test, y_test, verbose=0)
    val_score  = model2.evaluate(encoded_latent_val, y_val, verbose=0)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
    print('Test score:', test_score[0])
f.close()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 12)          0         
_________________________________________________________________
gru1 (GRU)                   (None, None, 32)          4320      
_________________________________________________________________
gru2 (GRU)                   (None, None, 16)          2352      
_________________________________________________________________
dense_9 (Dense)              (None, None, 16)          272       
_________________________________________________________________
de_gru1 (GRU)                (None, None, 16)          1584      
_________________________________________________________________
de_gru2 (GRU)                (None, None, 32)          4704      
_________________________________________________________________
dense_10 (Dense)             (None, None, 12)          396       
Total para

C:\Users\fengxd\Anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mean_squared_error,loss,mean_squared_error,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2256/2256 [==============================] - 1s 435us/step - loss: 14.9358 - mean_squared_error: 14.9358 - val_loss: 13.8534 - val_mean_squared_error: 13.8534
Epoch 3/20
2256/2256 [==============================] - 1s 444us/step - loss: 11.6265 - mean_squared_error: 11.6265 - val_loss: 12.6491 - val_mean_squared_error: 12.6491
Epoch 4/20
2256/2256 [==============================] - 1s 430us/step - loss: 9.4548 - mean_squared_error: 9.4548 - val_loss: 9.7723 - val_mean_squared_error: 9.7723
Epoch 5/20
2256/2256 [==============================] - 1s 430us/step - loss: 7.9652 - mean_squared_error: 7.9652 - val_loss: 9.2184 - val_mean_squared_error: 9.2184
Epoch 6/20
2256/2256 [==============================] - 1s 437us/step - loss: 7.0529 - mean_squared_error: 7.0529 - val_loss: 8.0432 - val_mean_squared_error: 8.0432
Epoch 7/20
2256/2256 [==============================] - 1s 441us/step - loss: 6.3581 - mean_squared_error: 6.3581 - val_loss: 7.4545 - val_mean_squared_error: 7.4545
Epoch 8

C:\Users\fengxd\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Users\fengxd\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 16)                2576      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 2,593
Trainable params: 2,593
Non-trainable params: 0
_________________________________________________________________
Train on 2256 samples, validate on 483 samples
Epoch 1/1
2256/2256 [==============================] - 2s 725us/step - loss: 2.4715 - mean_squared_error: 2.4715 - val_loss: 1.0153 - val_mean_squared_error: 1.0153
Test score: 1.1119917433701194
Train on 2256 samples, validate on 483 samples
Epoch 1/1
2256/2256 [==============================] - 0s 31us/step - loss: 0.9419 - mean_squared_error: 0.9419 - 

In [5]:
## LSTM SelfAttention

model = Sequential()
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE, input_shape=input_shape,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE,return_sequences=True)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Flatten()) #拉成一维数据
model.add(Dense(16, activation='relu')) 
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu')) #全连接层2
# model.add(Dense(1, activation='relu')) #全连接层2
 
#编译模型
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.0000004)
#训练模型
f = open('result_epoch_rnn_SA.csv','w')
for epoch in range(epochs):
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    test_score = model.evaluate(x_test, y_test, verbose=0)
    val_score  = model.evaluate(x_val, y_val, verbose=0)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
    print('Test score:', test_score[0])
f.close()

C:\Users\fengxd\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 22699 samples, validate on 4864 samples
Epoch 1/1
22699/22699 [==============================] - 13s 591us/step - loss: 1.3486 - mean_squared_error: 1.3486 - val_loss: 1.2828 - val_mean_squared_error: 1.2828


C:\Users\fengxd\Anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mean_squared_error,loss,mean_squared_error,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Test score: 1.1862049976200226
Train on 22699 samples, validate on 4864 samples
Epoch 1/1
22699/22699 [==============================] - 11s 479us/step - loss: 1.2559 - mean_squared_error: 1.2559 - val_loss: 1.2524 - val_mean_squared_error: 1.2524
Test score: 1.2478933780987076
Train on 22699 samples, validate on 4864 samples
Epoch 1/1
22699/22699 [==============================] - 11s 476us/step - loss: 1.2486 - mean_squared_error: 1.2486 - val_loss: 1.1736 - val_mean_squared_error: 1.1736
Test score: 1.119878930195403
Train on 22699 samples, validate on 4864 samples
Epoch 1/1
22699/22699 [==============================] - 11s 492us/step - loss: 1.2394 - mean_squared_error: 1.2394 - val_loss: 1.3421 - val_mean_squared_error: 1.3421
Test score: 1.2344725011872235
Train on 22699 samples, validate on 4864 samples
Epoch 1/1
22699/22699 [==============================] - 11s 491us/step - loss: 1.2330 - mean_squared_error: 1.2330 - val_loss: 1.2871 - val_mean_squared_error: 1.2871
Test scor

In [ ]:
## LSTM Bi
model = Sequential()
#model.add(keras.layers.Embedding(input_dim=INPUT_SIZE, output_dim=16,mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE, input_shape=input_shape,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE,return_sequences=True)))

model.add(Flatten()) #拉成一维数据
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='relu')) #全连接层2
# model.add(Dense(1, activation='relu')) #全连接层2
 
#编译模型
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00000001)
#训练模型
f = open(datasetName+'result_epoch_rnn_bi.csv','w')
for epoch in range(epochs):
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    test_score = model.evaluate(x_test, y_test, verbose=0)
    val_score  = model.evaluate(x_val, y_val, verbose=0)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
    print('Test score:', test_score[0])
f.close()

C:\Users\fengx\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 10s 496us/step - loss: 1.4462 - mse: 1.4462 - val_loss: 1.5436 - val_mse: 1.5436


C:\Users\fengx\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Test score: 1.5030113222088965
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 8s 379us/step - loss: 1.3329 - mse: 1.3329 - val_loss: 1.3620 - val_mse: 1.3620
Test score: 1.383026429693588
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 8s 386us/step - loss: 1.3121 - mse: 1.3121 - val_loss: 1.2835 - val_mse: 1.2835
Test score: 1.2879266967838272
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 8s 393us/step - loss: 1.3013 - mse: 1.3013 - val_loss: 1.2734 - val_mse: 1.2734
Test score: 1.2676718821438453
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 8s 396us/step - loss: 1.2941 - mse: 1.2941 - val_loss: 1.2952 - val_mse: 1.2952
Test score: 1.2968573546191804
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================

In [6]:
## mlp
model = Sequential()
model.add(Flatten()) #拉成一维数据
model.add(Dense(32, activation='relu')) 
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu')) #全连接层2
# model.add(Dense(1, activation='relu')) #全连接层2
 
#编译模型
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #打印模型

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000000005)
#训练模型
f = open(datasetName+'result_epoch_mlp.csv','w')
for epoch in range(epochs):
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    test_score = model.evaluate(x_test, y_test, verbose=0)
    val_score  = model.evaluate(x_val, y_val, verbose=0)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
    print('Test score:', test_score[0])
f.close()
model.summary() 

C:\Users\fengx\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 1s 48us/step - loss: 2.3459 - mse: 2.3459 - val_loss: 1.5719 - val_mse: 1.5719
Test score: 1.615163203508504
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
 3328/21015 [===>..........................] - ETA: 0s - loss: 1.6000 - mse: 1.6000

C:\Users\fengx\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


21015/21015 [==============================] - 1s 34us/step - loss: 1.6155 - mse: 1.6155 - val_loss: 1.4943 - val_mse: 1.4943
Test score: 1.4662687530405132
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 1s 34us/step - loss: 1.4961 - mse: 1.4961 - val_loss: 1.3866 - val_mse: 1.3866
Test score: 1.4101997326330595
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - ETA: 0s - loss: 1.4666 - mse: 1.466 - 1s 34us/step - loss: 1.4607 - mse: 1.4607 - val_loss: 1.4984 - val_mse: 1.4984
Test score: 1.5859291489929157
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 1s 36us/step - loss: 1.4229 - mse: 1.4229 - val_loss: 1.3904 - val_mse: 1.3904
Test score: 1.405143424840092
Train on 21015 samples, validate on 4476 samples
Epoch 1/1
21015/21015 [==============================] - 1s 35us/step - loss: 1.4082 - mse: 1.4082 - val_loss

In [2]:
import config_hep as cf

observation time 94607999
the number of time interval: 6
time interval: 15768000


In [7]:
y_train[1]

6.022367813028454